In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import pysift
from PIL import Image
from collections import Counter
from __future__ import division
import numpy as np
from skimage.metrics import structural_similarity as ssim
import math
from scipy import misc

In [ ]:
## Crop size
def cropImage(path, number, size):
    path = path+"/"+ str(number).zfill(size);
    img = cv.imread(path+".jpg",0)
    if(number > 2000):
        return img
    crop = open(path+ ".txt").read().split()
    x1 = int(crop[0])
    y1 = int(crop[1])
    x2 = x1 + int(crop[2])
    y2 = y1 + int(crop[3])
    img = img[y1:y2,x1:x2]
    #img = cv.fastNlMeansDenoising(img,None,3,7,21)
    #img = cv.GaussianBlur(img,(5,5),cv.BORDER_DEFAULT)
    return img
def getImage(path, number, size):
    path = path+"/"+ str(number).zfill(size);
    img = cv.imread(path+".jpg",0)
    return img
def changeToImg1Size(img1, img2):
    img2 = cv.resize(img2, (img1.shape[1],img1.shape[0]), interpolation = cv.INTER_AREA)
    return img2
# query = cropImage("Queries",1,2) #queuies data
# compare = cropImage("Images",686,4) #rank data
#changeToImg1Size(query, compare)

In [ ]:
sift = cv.SIFT_create()
def computeSIFT(image):
    return sift.detectAndCompute(image,None)

In [ ]:
def sift_knn(query, compare):
    kp1, des1 = computeSIFT(query)
    kp2, des2 = computeSIFT(compare)
    matches, good = calculateMatches(des1, des2)
    score = (len(good)/len(matches)) if len(good) > 0 else -10
    return score
def sift_BF(query, compare):
    kp1, des1 = computeSIFT(query)
    kp2, des2 = computeSIFT(compare)
    matches = calculateMatches2(des1, des2)
    score = 100 * (len(matches)/min(len(kp1),len(kp2)))
    return score

#sift_BF(1, 383)

In [ ]:
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 100)
flann = cv.FlannBasedMatcher(index_params, search_params)
def calculateMatches(des1,des2):
    matches = flann.knnMatch(des1,des2,k=2)
    topResults1 = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            topResults1.append([m])
    return matches, topResults1

In [ ]:
def SIFT_Method(query_lenght = 20, modSize = 5001): # sift
    #A text file records output
    text_file = open("rankListSIFT.txt", "w")
    allMatches = []
    #allSimilarity = []
    result_str = ""
    for i in range(1, query_lenght+1):
        query = cropImage("Queries",i,2) #queuies data
        
        result_str +="Q"+str(i)+":"
        for k in range(1,modSize):
            try:
                compare = cropImage("Images",k,4) #rank data
                compare = changeToImg1Size(query, compare)
                scores = sift_knn(query, compare)

                allMatches.append([k, scores])
                #print(k)
            except Exception:
                print("exception")
        allMatches.sort(key=lambda match: match[1], reverse=True)
        
        for id, _ in allMatches:
            result_str = result_str+" "+str(id)
        result_str += "\n"
        #allSimilarity.append[allMatches]
        allMatches = []
    text_file.write(result_str)

In [ ]:
def OpenCV_SSIM_Score(query, compare):
    compare_resize = changeToImg1Size(query, compare)
    query_edge = cv.Canny(query, 30, 150)
    compare_edge = cv.Canny(compare_resize, 30, 150)
    score, _= ssim(query_edge, compare_edge, win_size=101, full=True)
    return score

In [ ]:
def SSIM_Mean(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()


def calculate_ssim(img1, img2):
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return SSIM_Mean(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return SSIM_Mean(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')

In [ ]:
def SSIM_Method(query_lenght = 20, modSize = 5001):
    text_file = open("rankListSSIM.txt", "w")
    allMatches = []
    modSize = 5001
    result_str = ""
    for i in range(1,query_lenght+1):
        query = cropImage("Queries",i,2) #queuies data
        result_str +="Q"+str(i)+":"
        print(i)
        for k in range(1,modSize):
            compare = cropImage("Images",k,4) #rank data
            compare_resize = changeToImg1Size(query, compare)
            query_edge = cv.Canny(query, 30, 150)
            compare_edge = cv.Canny(compare_resize, 30, 150)
            score = calculate_ssim(query_edge, compare_edge)
            allMatches.append([k, score])
            #print(k)
        allMatches.sort(key=lambda match: match[1], reverse=True)
        #printResult(allMatches, query)
        for id, _ in allMatches:
            result_str = result_str+" "+str(id)
        result_str += "\n"
        allMatches = []
    text_file.write(result_str)

In [ ]:
# average precision
def ave_pre(array):
    ap = 0
    for idx, m in enumerate(array):
        ap_m = (idx + 1) / (m + 1)
        ap = ap + ap_m
    ap = ap / len(array)
    return ap 
def MAP(path):
    # read ground truth num of the relevant images
    with open('./rank_groundtruth.txt') as f:
        rank_line = f.read().splitlines()
    # read retrieved num of the relevant images
    with open('./'+path+'.txt') as f:
        rank_result = f.read().splitlines()

    # compute mean average precision
    ap_sum = 0
    map_result = []
    for idx, line in enumerate(rank_result):
        line_str = line.split()
        query_num = int(line_str[0][1]) - 1
        result_num = [int(x) for x in line_str[1:]]
        
        rank_str = rank_line[idx].split()
        rank_gt = [int(x) for x in rank_str[1:]]
        find_idx = []
        for num in rank_gt:
            ind = np.where(np.array(result_num) == num)
            find_idx.extend(ind)
        find_idx = np.array(find_idx).reshape(len(find_idx),)
        find_idx = np.sort(find_idx)
        ap = ave_pre(find_idx)
        map_result.append(ap)
        ap_sum = ap_sum + ap
    return map_result

In [ ]:
def calculateFinalSimilarity():
    map_result = [MAP("rankListSIFT"), MAP("rankListSSIM")]
    with open('./rankListSIFT.txt') as f:
        rank_result_sift = f.read().splitlines()
    with open('./rankListSSIM.txt') as f:
        rank_result_ssim = f.read().splitlines()
    finalResult = []
    for idx in range(len(rank_result_sift)):
        #split sift result to list
        line_str_sift = rank_result_sift[idx].split()
        result_num_sift = [int(x) for x in line_str_sift[1:]]
        #split ssim result to list
        line_str_ssim = rank_result_sift[idx].split()
        result_num_ssim = [int(x) for x in line_str_ssim[1:]]
        
        sift_map = map_result[0][idx]
        ssim_map = map_result[1][idx]
        sum_map = sift_map+ssim_map
        ratio_sift = sift_map / sum_map
        ratio_ssim = ssim_map / sum_map
        # list for sorting result
        scores = []
        for i in range(1,5001):
            scores.append([i,0])
        for i in range(len(result_num_sift)):
            rank = (i+1)
            scores[result_num_sift[i]-1][1] += ratio_sift * rank
            scores[result_num_ssim[i]-1][1] += ratio_ssim * rank
        
        scores.sort(key=lambda match: match[1], reverse=False)
        tempResults = []
        for i in range(5000):
            tempResults.append(scores[i][0]) 
        finalResult.append(tempResults)
    #MAP_2(finalResult)
    write_rankList(finalResult)
    
def write_rankList(result, path = "rankList"):
    text_file = open(path+".txt", "w")
    result_str = ""
    
    for idx, result_num in enumerate(result):
        result_str += "Q"+str(idx+1)+":"
        for rank in result_num:
            result_str+= " "+str(rank)
        result_str += "\n"
    text_file.write(result_str)
    
def MAP_2(result):
    # read ground truth num of the relevant images
    with open('./rank_groundtruth.txt') as f:
        rank_line = f.read().splitlines()
    # compute mean average precision
    ap_sum = 0
    for idx, result_num in enumerate(result):
        rank_str = rank_line[idx].split()
        rank_gt = [int(x) for x in rank_str[1:]]
        find_idx = []
        #print(result_num)
        for num in rank_gt:
            ind = np.where(np.array(result_num) == num)
            find_idx.extend(ind)
        find_idx = np.array(find_idx).reshape(len(find_idx),)
        find_idx = np.sort(find_idx)
        ap = ave_pre(find_idx)
        print ("Average Precision of Q%d: %.4f"%(idx+1, ap))
        ap_sum = ap_sum + ap
    print ("Mean Average Precision: %f" %(ap_sum / len(rank_result)))
    

In [ ]:
def drawResult():
    with open("rankList.txt") as f:
        rank_result = f.read().splitlines()

    fig=plt.figure(figsize=(100, 100))
    for idx in range(10):
        line_str = rank_result[idx].split()
        result_num = [int(x) for x in line_str[1:]]
        images = []
        #x_offset = 100
        for k in range(10):
            path = "./Images/"+ str(result_num[k]).zfill(4)+".jpg";
            img = cv.imread(path)
            x = img.shape[1]
            y = img.shape[0]
            size = 300
            if x > y:
                cv.resize(img, (size, int(y * (size/x))))
            else:
                cv.resize(img, (int(x * (size/y)), size))
            row = idx
            col = k+1
            fig.add_subplot(10, 10, (row*10 + col))
            plt.imshow(img)
    plt.savefig("top10_results.png")
    plt.show()    

In [ ]:
if __name__ == '__main__':
    ql = 10
    SIFT_Method(query_length = ql)
    SSIM_Method(query_length = ql)
    calculateFinalSimilarity()
    drawResult()